In [51]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from langchain. prompts import PromptTemplate
import pinecone



In [ ]:
# create pinecone cluster (we use pincone vector db)
#generate api key from pinecone website
from dotenv import load_dotenv
import os

load_dotenv()
PINECONE_API_KEY  = os.getenv("PINECONE_API_KEY")
#PINECONE_API_ENV = "gcp-starter"
print(PINECONE_API_KEY) 

In [ ]:
#Extract data from the PDF
#only when we have more than one .pdf 

#from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
#def load_pdf(data):
#    loader = DirectoryLoader(data,
#                    glob="*.pdf",
#                    loader_cls=PyPDFLoader)
    
#    documents = loader.load()

#    return documents
#extracted_data = load_pdf(r"C:\Users\Sniper\AI Project\Chatbot-with-Llma-2\data\Medical_book.pdf")

In [53]:
from langchain_community.document_loaders import PyPDFLoader
#Extract data from the PDF
def load_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents

extracted_data = load_pdf(r"C:\Users\Sniper\AI Project\Chatbot-with-Llma-2\data\Medical_book.pdf")

In [ ]:
extracted_data

In [55]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [56]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5859


In [ ]:
# this 5859 chunks need to store in vector db
# convert to vector representation

text_chunks


# text_chunks


In [58]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [59]:
embeddings = download_hugging_face_embeddings()

In [60]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [61]:
query_result = embeddings.embed_query("Hello world")
print("Length (vector representation of Hello world(dimention)): ", len(query_result))

Length (vector representation of Hello world(dimention)):  384


In [62]:
# we are able to convert text to embedding (vectors) & dimention of the vector is 384
query_result

[-0.03447727486491203,
 0.03102317824959755,
 0.006734970025718212,
 0.026108985766768456,
 -0.03936202451586723,
 -0.16030244529247284,
 0.06692401319742203,
 -0.006441489793360233,
 -0.0474504791200161,
 0.014758856035768986,
 0.07087527960538864,
 0.05552763119339943,
 0.019193334504961967,
 -0.026251312345266342,
 -0.01010954286903143,
 -0.02694045566022396,
 0.022307461127638817,
 -0.022226648405194283,
 -0.14969263970851898,
 -0.017493007704615593,
 0.00767625542357564,
 0.05435224249958992,
 0.0032543970737606287,
 0.031725890934467316,
 -0.0846213847398758,
 -0.02940601296722889,
 0.05159561336040497,
 0.04812406003475189,
 -0.0033148222137242556,
 -0.058279167860746384,
 0.04196927323937416,
 0.022210685536265373,
 0.1281888335943222,
 -0.022338971495628357,
 -0.011656315997242928,
 0.06292839348316193,
 -0.032876335084438324,
 -0.09122604131698608,
 -0.031175347045063972,
 0.0526994913816452,
 0.04703482985496521,
 -0.08420311659574509,
 -0.030056199058890343,
 -0.02074483036

In [63]:
for i, val in enumerate(query_result):
    print(f"Dimension {i}: {val}")

Dimension 0: -0.03447727486491203
Dimension 1: 0.03102317824959755
Dimension 2: 0.006734970025718212
Dimension 3: 0.026108985766768456
Dimension 4: -0.03936202451586723
Dimension 5: -0.16030244529247284
Dimension 6: 0.06692401319742203
Dimension 7: -0.006441489793360233
Dimension 8: -0.0474504791200161
Dimension 9: 0.014758856035768986
Dimension 10: 0.07087527960538864
Dimension 11: 0.05552763119339943
Dimension 12: 0.019193334504961967
Dimension 13: -0.026251312345266342
Dimension 14: -0.01010954286903143
Dimension 15: -0.02694045566022396
Dimension 16: 0.022307461127638817
Dimension 17: -0.022226648405194283
Dimension 18: -0.14969263970851898
Dimension 19: -0.017493007704615593
Dimension 20: 0.00767625542357564
Dimension 21: 0.05435224249958992
Dimension 22: 0.0032543970737606287
Dimension 23: 0.031725890934467316
Dimension 24: -0.0846213847398758
Dimension 25: -0.02940601296722889
Dimension 26: 0.05159561336040497
Dimension 27: 0.04812406003475189
Dimension 28: -0.003314822213724255

In [66]:
from pinecone import ServerlessSpec 

index_name = "chatbot-llama-2"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [70]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

In [ ]:
#If we already have an index we can load it like this
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(id='3ff5c515-668a-4623-965b-4d3a879bdd81', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 135.0, 'page_label': '136', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\Users\\Sniper\\AI Project\\Chatbot-with-Llma-2\\data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of'), Document(id='8e60b4c1-e098-4977-9638-26f3d38a86f5', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00',

In [74]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [75]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [77]:
llm=CTransformers(model=r"C:\Users\Sniper\AI Project\Chatbot-with-Llma-2\model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [78]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [2]:
while True:
    user_input=input(f"Input Prompt:")
    result=({"query": user_input})
    print("Response : ", result["result"])

KeyError: 'result'